In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

from paus_utils import w_central
from jpasLAEs.utils import flux_to_mag

In [ ]:
# Load the photometry catalog
from load_paus_cat import load_paus_cat

cat_list = []
for field_name in ['W1', 'W2', 'W3']:
    path_to_cat = [f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_3arcsec_{field_name}_extinction_corrected.pq']
    cat = load_paus_cat(path_to_cat)

    mask_NB_number = (cat['NB_number'] > 39)
    cat['flx'] = cat['flx'][:, mask_NB_number]
    cat['err'] = cat['err'][:, mask_NB_number]
    cat['NB_mask'] = cat['NB_mask'][:, mask_NB_number]
    for key in cat.keys():
        if key in ['flx', 'err', 'NB_mask', 'area']:
            continue
        cat[key] = cat[key][mask_NB_number]

    stack_nb_ids = np.arange(12, 26 + 1)
    synth_BB_flx = np.average(cat['flx'][stack_nb_ids],
                            weights=cat['err'][stack_nb_ids] ** -2,
                            axis=0)

    cat['synth_r_mag'] = flux_to_mag(synth_BB_flx, w_central[-4])
    cat['r_mag'] = flux_to_mag(cat['flx'][-4], w_central[-4])

    cat_list.append(cat)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

for cat_i, cat in enumerate(cat_list):
    for nb_i in np.arange(0, 20):
        p16, p50, p84 = np.nanpercentile(cat['flx'][nb_i],
                                         [16, 50, 84])

        ax.errorbar(nb_i + 0.1 * cat_i, p50,
                    yerr=[[p84 - p50], [p50 - p16]],
                    fmt='s', capsize=1.5,
                    mfc=f'C{cat_i}', mec='k', ecolor=f'C{cat_i}')

    ax.errorbar([], [], marker='s', mfc=f'C{cat_i}', mec='k',
                ls='', label=f'W{cat_i + 1}')

ax.legend()

plt.show()